In [ ]:
from obspy.core import *
from matplotlib import pyplot as plt

from scipy.stats import norm
import numpy as np

In [ ]:
data_path = '/Volumes/Hao/data/067/YA/'
tr = read(data_path+'R1080/*DPZ*').decimate(factor=5, strict_length=False)

In [ ]:
tr_new = tr.copy()
t0 = UTCDateTime('2020-03-07T17:34:40Z')
tr_new.decimate(factor=5, strict_length=False)
tr_new = tr_new.slice(t0-600,t0+4200)
tr_new.filter('bandpass', freqmin=1, freqmax=7, zerophase=True)
tr_new.plot()

In [ ]:
tr1 = tr_new.slice(t0-600,t0+600)
tr2 = tr_new.slice(t0+600,t0+1800)
tr3 = tr_new.slice(t0+1800,t0+3000)
tr4 = tr_new.slice(t0+3000,t0+4200)

data1 = tr1[0].data
data2 = tr2[0].data
data3 = tr3[0].data
data4 = tr4[0].data


plt.figure(figsize=(20,15))
plt.subplot(3,2,1)
plt.specgram(data1,Fs=100,scale='linear')
plt.ylim([0,10])
plt.xlabel('Time/s')
plt.ylabel('Frequency/Hz')

plt.subplot(3,2,2)
plt.specgram(data2,Fs=100,scale='linear')
plt.ylim([0,10])
plt.xlabel('Time/s')
plt.ylabel('Frequency/Hz')

plt.subplot(3,2,3)
plt.specgram(data3,Fs=100,scale='linear')
plt.ylim([0,10])
plt.xlabel('Time/s')
plt.ylabel('Frequency/Hz')

plt.subplot(3,2,4)
plt.specgram(data4,Fs=100,scale='linear')
plt.ylim([0,10])
plt.xlabel('Time/s')
plt.ylabel('Frequency/Hz')

plt.subplot(3,2,5)
psd1 = plt.psd(data1, Fs=100)
psd2 = plt.psd(data2, Fs=100)
psd3 = plt.psd(data3, Fs=100)
psd4 = plt.psd(data4, Fs=100)
plt.legend(['1','2','3','4'])
plt.xlim([0,10])
plt.ylim([-20,50])

plt.show()

In [ ]:
r1 = 10*np.log10(psd1[0][8:14].sum()/psd2[0][8:14].sum())
r2 = 10*np.log10(psd2[0][8:14].sum()/psd3[0][8:14].sum())
r3 = 10*np.log10(psd3[0][8:14].sum()/psd4[0][8:14].sum())

r4 = 10*np.log10(psd1[0][15:25].sum()/psd2[0][15:25].sum())
r5 = 10*np.log10(psd2[0][15:25].sum()/psd3[0][15:25].sum())
r6 = 10*np.log10(psd3[0][15:25].sum()/psd3[0][15:25].sum())
print(r1-r4, r2-r5, r3-r6)

In [ ]:
event_list = open('../pick/train.gmt').readlines()
mar7 = []
for k in event_list:
    if k[9] >= '7':
        mar7.append(UTCDateTime(k[:-1]))   

In [ ]:
t0 = UTCDateTime('2020-03-07T00:00:00Z')
k,n = 0,0
ratios = []
while t0 < UTCDateTime('2020-03-07T23:40:00Z'):
    if t0+1800 > mar7[k]:
        t0 += 2400
        k += 1
    else:
        n += 1
        tr_new = tr.copy()
        tr_new = tr_new.slice(t0-600,t0+1800)
        tr_new.filter('bandpass', freqmin=1, freqmax=5, zerophase=True)
    
        tr1 = tr_new.slice(t0-600,t0+600)
        tr2 = tr_new.slice(t0+600,t0+1800)
        data1 = tr1[0].data
        data2 = tr2[0].data
        psd1 = plt.psd(data1, Fs=100)
        psd2 = plt.psd(data2, Fs=100)
    
        r1 = 10*np.log10(psd1[0][8:14].sum()/psd2[0][8:14].sum())
        r4 = 10*np.log10(psd1[0][15:25].sum()/psd2[0][15:25].sum())
        ratio = r1 - r4
        ratios.append(ratio)
        
        t0 += 10
        print(n, ratio)

In [ ]:
for k in range(len(mar7)):
    t0 = mar7[k]
    tr_new = tr.copy()
    tr_new = tr_new.slice(t0-600,t0+1800)
    tr_new.filter('bandpass', freqmin=1, freqmax=5, zerophase=True)
    
    tr1 = tr_new.slice(t0-600,t0+600)
    tr2 = tr_new.slice(t0+600,t0+1800)
    data1 = tr1[0].data
    data2 = tr2[0].data
    psd1 = plt.psd(data1, Fs=100)
    psd2 = plt.psd(data2, Fs=100)
    
    r1 = 10*np.log10(psd1[0][8:14].sum()/psd2[0][8:14].sum())
    r4 = 10*np.log10(psd1[0][15:25].sum()/psd2[0][15:25].sum())
    print(r1-r4)
    
    

In [ ]:
mu, sigma = np.mean(ratios), np.std(ratios)
n, bins, patches = plt.hist(ratios, bins=50, density=True, alpha=0.5)

x = np.linspace(-10, 10, 100)
plt.plot(x, norm.pdf(x, mu, sigma))
plt.show()

In [ ]:
mu, sigma